# Imports

In [24]:
import pandas as pd
import numpy as np 
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
import warnings

# Read the dataframe

In [25]:
file_path = r"C:\Users\karth\Downloads\operational_demand.csv"
df = pd.read_csv(file_path)

# Data Cleaning & Manipulation

In [26]:
df = df.drop(['awbiMetadataKeyHash', 'awbiMetadataChangeHash','awbiMetadataCreateDateTime',
                                'awbiMetadataLoadYear','awbiMetadataLoadMonth','awbiMetadataLoadDay',
                                'awbiMetadataLoadHour','awbiMetadataLoadDateTime'], axis=1)
df = df.reset_index(drop=True)

In [27]:
unique_value = df['operationalDemandWorkCentreGroup'].nunique()
print("Unique values in operationalDemandWorkCentreGroup: ", unique_value)

value_counts = df['operationalDemandWorkCentreGroup'].value_counts()
print("Value counts in operationalDemandWorkCentreGroup: ", value_counts)

Unique values in operationalDemandWorkCentreGroup:  1
Value counts in operationalDemandWorkCentreGroup:  operationalDemandWorkCentreGroup
493    6035
Name: count, dtype: int64


In [28]:
df.dtypes

operationalDemandDate                  object
operationalDemandOrderTypeGroup        object
operationalDemandProcess               object
operationalDemandWorkCentreGroup        int64
operationalDemandOpHoursCount           int64
operationalDemandOpHoursSum           float64
operationalDemandDateSid                int64
operationalDemandDateWeekBeginDate     object
operationalDemandSumZeroedRe          float64
operationalDemandOpTravelHours        float64
dtype: object

In [29]:
# Changing the operationalDemandDate column to a date time format allowing us to grab features from the date when required
df['operationalDemandDate'] = pd.to_datetime(df['operationalDemandDate'])

#Changing the columns below to a string/flot data type respecriveely to allow for data manipulation
df['operationalDemandWorkCentreGroup'] = df['operationalDemandWorkCentreGroup'].astype(str)
df['operationalDemandDateSid'] = df['operationalDemandDateSid'].astype(str)
df['operationalDemandOpTravelHours'] = df['operationalDemandOpTravelHours'].astype(float)

In [30]:
#Sorting the dates in an ascending order
df = df.sort_values(by=['operationalDemandDateWeekBeginDate','operationalDemandDate'], ascending= True)

In [31]:
# Making use of the dt.year/month etc repectively to create a new column in the dataset which stores only the year/month of the operationalDemandDate
df['OperationalDemandYear'] = df['operationalDemandDate'].dt.year
df['OperationalDemandMonth'] = df['operationalDemandDate'].dt.month
df['OperationalDemandDayofweek'] = df['operationalDemandDate'].dt.day_of_week
df['OperationalDemandQuarter'] = df['operationalDemandDate'].dt.quarter
df

,operationalDemandDate,operationalDemandOrderTypeGroup,operationalDemandProcess,operationalDemandWorkCentreGroup,operationalDemandOpHoursCount,operationalDemandOpHoursSum,operationalDemandDateSid,operationalDemandDateWeekBeginDate,operationalDemandSumZeroedRe,operationalDemandOpTravelHours,OperationalDemandYear,OperationalDemandMonth,OperationalDemandDayofweek,OperationalDemandQuarter
1158,2019-04-01,PLANNED,COLLECTION,493,140,83.700000,20190401,2019-04-01T00:00:00.000+0000,47.266667,54.224462,2019,4,0,2
4735,2019-04-01,PLANNED,TREATMENT,493,30,80.800000,20190401,2019-04-01T00:00:00.000+0000,47.666667,13.086870,2019,4,0,2
5039,2019-04-01,REACTIVE,MAINTENANCE,493,39,87.066667,20190401,2019-04-01T00:00:00.000+0000,59.500000,21.249216,2019,4,0,2
2338,2019-04-02,PLANNED,COLLECTION,493,1,0.500000,20190402,2019-04-01T00:00:00.000+0000,0.416667,0.238611,2019,4,1,2
3217,2019-04-02,REACTIVE,COLLECTION,493,20,38.233333,20190402,2019-04-01T00:00:00.000+0000,26.333333,8.394569,2019,4,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3400,2024-01-29,PLANNED,COLLECTION,493,1,2.000000,20240129,2024-01-29T00:00:00.000+0000,0.000000,0.587613,2024,1,0,1
6028,2024-01-29,PLANNED,MAINTENANCE,493,129,118.083333,20240129,2024-01-29T00:00:00.000+0000,0.000000,67.171177,2024,1,0,1
254,2024-02-01,PLANNED,COLLECTION,493,120,86.883333,20240201,2024-01-29T00:00:00.000+0000,0.000000,70.513571,2024,2,3,1
5305,2024-02-01,PLANNED,MAINTENANCE,493,11,11.000000,20240201,2024-01-29T00:00:00.000+0000,0.000000,5.727775,2024,2,3,1


- **For OperationalDemandDayofweek**
- Monday = 0
- Tuesday = 1
- Wednesday = 2
- Thursday = 3
- Friday = 4
- Saturday = 5
- Sunday = 6

In [32]:
#Looking at the minimum date from when the forecasting data starts
print("The minimum date from when the forecasting data starts is: ", df['operationalDemandDate'].min())

#Looking at the minimum date from when the forecasting data ends
print("The maximum date from when the forecasting data ends is: ", df['operationalDemandDate'].max())

The minimum date from when the forecasting data starts is:  2019-04-01 00:00:00
The maximum date from when the forecasting data ends is:  2024-02-04 00:00:00


# Handling the outliers

In [33]:
#USing standard anomaly detection of q1/q3 +- 1.5 * IQR to identify outliers and improve the predictability of models

#Upper and Lower Quartiles for the flow data to identify IQR
q1_hours = df['operationalDemandOpHoursSum'].quantile(0.25)
q3_hours = df['operationalDemandOpHoursSum'].quantile(0.75)

#IQR and Upper and lower bounds to catch outliers 
iqr_hours = q3_hours - q1_hours
lower_bound_hours = q1_hours - 1.5 * iqr_hours
upper_bound_hours = q3_hours + 1.5 * iqr_hours


#Filtered Data frame to apply boundaries to values for each site and for the respective feature and response variables
df = df[
    (df['operationalDemandOpHoursSum'] >= lower_bound_hours) & (df['operationalDemandOpHoursSum'] <= upper_bound_hours)]

# Differntiating Weekday/Weekened

In [35]:
# Filtering a weekday df to only contain days from mon - fri
weekday_df = df[df['OperationalDemandDayofweek'].isin([0, 1, 2, 3, 4])]
weekday_df['OperationalDemandDayofweek'].unique()

array([0, 1, 2, 3, 4])

# Weekened Manipulation 

In [36]:
# Weeknd data set to only include sat - sun
weekend_df = df[df['OperationalDemandDayofweek'].isin([5, 6])]
weekend_df

,operationalDemandDate,operationalDemandOrderTypeGroup,operationalDemandProcess,operationalDemandWorkCentreGroup,operationalDemandOpHoursCount,operationalDemandOpHoursSum,operationalDemandDateSid,operationalDemandDateWeekBeginDate,operationalDemandSumZeroedRe,operationalDemandOpTravelHours,OperationalDemandYear,OperationalDemandMonth,OperationalDemandDayofweek,OperationalDemandQuarter
1812,2019-04-06,REACTIVE,?,493,1,0.416667,20190406,2019-04-01T00:00:00.000+0000,1.416667,0.596775,2019,4,5,2
4362,2019-04-06,REACTIVE,TREATMENT,493,9,10.333333,20190406,2019-04-01T00:00:00.000+0000,4.916667,2.705398,2019,4,5,2
1299,2019-04-07,REACTIVE,?,493,5,5.166667,20190407,2019-04-01T00:00:00.000+0000,6.666667,2.983873,2019,4,6,2
2297,2019-04-07,PLANNED,MAINTENANCE,493,2,5.533333,20190407,2019-04-01T00:00:00.000+0000,2.000000,2.146389,2019,4,6,2
3402,2019-04-07,REACTIVE,COLLECTION,493,15,32.516667,20190407,2019-04-01T00:00:00.000+0000,16.500000,7.100452,2019,4,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5924,2023-12-24,PLANNED,TREATMENT,493,18,9.000000,20231224,2023-12-18T00:00:00.000+0000,0.000000,17.457164,2023,12,6,4
625,2024-01-06,PLANNED,COLLECTION,493,1,1.000000,20240106,2024-01-01T00:00:00.000+0000,0.000000,0.587613,2024,1,5,1
5765,2024-01-14,PLANNED,COLLECTION,493,2,1.000000,20240114,2024-01-08T00:00:00.000+0000,0.000000,1.175226,2024,1,6,1
4363,2024-01-28,PLANNED,COLLECTION,493,1,4.000000,20240128,2024-01-22T00:00:00.000+0000,0.000000,0.587613,2024,1,6,1


In [38]:
# Weekend groupby in which we get a sum of hours for the weekend
weekend_groupby = weekend_df.groupby('operationalDemandDateWeekBeginDate')['operationalDemandOpHoursSum'].sum().reset_index()
weekend_groupby

,operationalDemandDateWeekBeginDate,operationalDemandOpHoursSum
0,2019-04-01T00:00:00.000+0000,90.816667
1,2019-04-08T00:00:00.000+0000,79.333333
2,2019-04-15T00:00:00.000+0000,113.766667
3,2019-04-22T00:00:00.000+0000,38.583333
4,2019-04-29T00:00:00.000+0000,57.300000
...,...,...
245,2023-12-18T00:00:00.000+0000,11.250000
246,2024-01-01T00:00:00.000+0000,1.000000
247,2024-01-08T00:00:00.000+0000,1.000000
248,2024-01-22T00:00:00.000+0000,4.000000


# Weekday Manipulation

In [39]:
# grouping by the week and the operational demand sum to get a sum for the week
week_groupby = weekday_df.groupby('operationalDemandDateWeekBeginDate')['operationalDemandOpHoursSum'].sum().reset_index()

# Alligning the dates for both week and weekend to ensure both df's have the same date time frame
week_groupby = week_groupby.drop(index=236, axis=1)
week_groupby

,operationalDemandDateWeekBeginDate,operationalDemandOpHoursSum
0,2019-04-01T00:00:00.000+0000,680.300000
1,2019-04-08T00:00:00.000+0000,501.683333
2,2019-04-15T00:00:00.000+0000,583.566667
3,2019-04-22T00:00:00.000+0000,346.100000
4,2019-04-29T00:00:00.000+0000,648.450000
...,...,...
247,2024-01-01T00:00:00.000+0000,2.000000
248,2024-01-08T00:00:00.000+0000,0.500000
249,2024-01-15T00:00:00.000+0000,87.250000
250,2024-01-22T00:00:00.000+0000,5.633333


# Shift Function for Week

In [40]:
# Shift function to create date time features in which we can utlise previous weeks data as covariates to predict workload 
def shift_function(df,new_column_name,shift):
  df[new_column_name] = df['operationalDemandOpHoursSum'].shift(shift)

# previous week's data ranging from week 1 - 4
shift_1_week = shift_function(week_groupby, 'prev_week_1', 1)
shift_2_week = shift_function(week_groupby, 'prev_week_2', 2)
shift_3_week = shift_function(week_groupby, 'prev_week_3', 3)
shift_4_week = shift_function(week_groupby, 'prev_week_4', 4)

In [41]:
# The shift function creates na within the df in which need to be dropped
week_groupby = week_groupby.dropna(axis=0)